# Extending the pre-night dashboard with a tab with a `matplotlib` plot

## Notebook perparation

### Load jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import importlib
import warnings
from astropy.time import Time
import pandas as pd
import panel as pn
import param
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import schedview.collect.opsim
import schedview.app
import schedview.app.prenight

### Further preparation of the notebook

Configure the notebook to show `panel` plots and dashboards:

In [ ]:
pn.extension("terminal")

Configure the notebook *not* to show `matplotlib` plots using usual jupyter `matplotlib` backend, because `panel` will be doing this and we don't want duplicate plots.

In [ ]:
%matplotlib agg

## Create the extended dashboard app

Create a function that takes a `pandas.DataFrame` of visits, and returns a `matplotlib.figure` with the plot we want:

In [ ]:
def create_cumm_az_plot(visits):
    fig, axes = plt.subplots(3, sharex=True, gridspec_kw={"hspace": 0})
    axes[0].plot(visits.start_date, visits.cummTelAz)
    axes[0].set_ylabel("Cumulative az rot.")
    axes[1].plot(visits.start_date, visits.azimuth)
    axes[1].set_ylabel("Field Az")
    axes[2].plot(visits.start_date, visits.slewTime)
    axes[2].set_ylabel("Slew time")
    axes[2].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
    axes[2].set_xlabel("Time (UTC)")
    return fig

Subclass `schedview.app.prenight.prenight.Prenight` which overrides or add three new elements:

1. Override the `shown_tabs` class member in `Prenight` to include your new plot in the list of tabs. The `objects` argument of `param.ListSelector` includes the names of all tabs which can be included when creating a dashboard, and the `default` argument lists the tabs that will be shown if your subclass is instantiated without the `shown_tabs` argument being set.
2. Create a method that returns an instance of `panel.Pane` what wraps a call to the function we made above. Wrap it with a `@param.depends` decorator so that the plot will get updated when new visits are loaded.
3. Replace `initialize_tab_contents` from the parent class with an implementation that includes the new plot.

In [ ]:
class PrenightWithAzWrapTab(schedview.app.prenight.prenight.Prenight):
    # Define which tabs should be shown.
    # The name for your new tab should be included in
    # both the default and objects list to be visible
    # by default.
    shown_tabs = param.ListSelector(
        default=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "Azimuth wrap",
        ],
        objects=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "Visit explorer",
            "Azimuth wrap",
        ],
        doc="The names of the tabs to show.",
    )

    # If you plot needs to be update when the data is changed
    # e.g. if a new set of data is loaded, list the member
    # in the @param.depends decorator
    @param.depends(
        "_visits",
    )
    def make_az_wrap_plot(self):
        # The dashboard may not have any loaded visits, for example
        # when first loaded, so handle that situation gracefully
        if self._visits is None:
            return "No visits are loaded"

        return pn.pane.Matplotlib(create_cumm_az_plot(self._visits))

    def initialize_tab_contents(self):
        # Start with the dictionary with the tabs defined
        # in base class.
        tab_contents = super().initialize_tab_contents()

        # Add your new plot to this dictionary
        new_tab_name = "Azimuth wrap"
        tab_contents[new_tab_name] = pn.param.ParamMethod(
            self.make_az_wrap_plot, loading_indicator=True
        )

        return tab_contents

## Try out our new dashboard

Load some sample data:

In [ ]:
sample_data_dir = importlib.resources.files("schedview").joinpath("data")
sample_opsim_db = str(sample_data_dir.joinpath("sample_opsim.db"))
sample_scheduler_pickle = str(sample_data_dir.joinpath("sample_scheduler.pickle.xz"))
sample_rewards_h5 = str(sample_data_dir.joinpath("sample_rewards.h5"))

Get the date the sample visits start on:

In [ ]:
def get_sim_start_date(opsim_fname):
    opsim = schedview.collect.opsim.read_opsim(opsim_fname)
    start_mjd = opsim.observationStartMJD.min()
    start_datetime_utc = Time(start_mjd, format="mjd").datetime
    night_date = (
        pd.Timestamp(start_datetime_utc, tz="UTC")
        .tz_convert("Chile/Continental")
        .date()
    )
    return night_date


night_date = get_sim_start_date(sample_opsim_db)
night_date

Actually show our dashboard:

In [ ]:
prenight = PrenightWithAzWrapTab()
pn_app = prenight.make_app(
    night_date,
    opsim_db=sample_opsim_db,
    scheduler=sample_scheduler_pickle,
    rewards=sample_rewards_h5,
)
pn_app

## Other data

To explore what data is available to be used in plots, data from (or derived from) the other loaded files can be used as well.

### Visits (`_visits` and `_visits_cds`)

The `_visits` parameter contains a `pandas.DataFrame` of the visits loaded from the opsim database for the specified night:

In [ ]:
type(prenight._visits)

In [ ]:
prenight._visits

The `_visits_cds` member contains the same data, in an instance of `panel.models.ColumenDataSource`.
When making `bokeh` plots, `_visits_cds` has the advantage of supporting `bokeh`'s automatic linking with other plots that also use `_visits_cds`, for example linked brushing.

In [ ]:
type(prenight._visits_cds)

In [ ]:
prenight._visits_cds.data.keys()

### The instance of the scheduler

The instance of the scheduler loaded form the pickle is referenced by the `_scheduler` parameter:

In [ ]:
type(prenight._scheduler)

### Alamanc events

The `_alamanac_evests` parameter holds almanac events, computed by the scheduler for the specified nights:

In [ ]:
prenight._almanac_events

### Reward data

The `_reward_df` parameter supplies rewards for each call to the scheduler:

In [ ]:
prenight._reward_df

The `_obs_rewards` parameter connects these calls to specific obserations in the visits table: the index corresponds to the `observationStartMJD` column in the `visits` `DataFrame`, and the value to the `queue_fill_mjd_ns` column in `_reward_df`.
So, to show the statistics for rewards calculated in the call where the 100th observation was chosen:

In [ ]:
my_obs = prenight._visits.loc[100]
my_obs.to_frame().T

In [ ]:
queue_fill_mjd_ns = prenight._obs_rewards[my_obs.observationStartMJD]
obs_rewards = prenight._reward_df.query(f"queue_fill_mjd_ns=={queue_fill_mjd_ns}")
obs_rewards

### Using parameters

To use these parameters in a method, just use the members directly in calling the code, and use the `param.depends` decorator to let `panel` know it needs to be updated when those data are updated.

For example, if you created a function `my_plot` that makes a matplotlib figure and takes insntances of `_visits`, `_reward_df`, and `_obs_rewards` as arguments, your subclass might look like this:

In [ ]:
class PrenightWithMyTab(schedview.app.prenight.prenight.Prenight):
    # Define which tabs should be shown.
    # The name for your new tab should be included in
    # both the default and objects list to be visible
    # by default.
    shown_tabs = param.ListSelector(
        default=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "My plot",
        ],
        objects=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "Visit explorer",
            "My plot",
        ],
        doc="The names of the tabs to show.",
    )

    # If you plot needs to be update when the data is changed
    # e.g. if a new set of data is loaded, list the member
    # in the @param.depends decorator
    @param.depends("_visits", "_reward_df", "_obs_rewards")
    def make_my_plot(self):
        # The dashboard may not have any loaded visits, for example
        # when first loaded, so handle that situation gracefully
        if self._visits is None:
            return "No visits are loaded"

        if self._reward_df is None or self._obs_rewards is None:
            return "No rewards are loaded"

        return pn.pane.Matplotlib(
            my_plot(self._visits, self._reward_df, self._obs_rewards)
        )

    def initialize_tab_contents(self):
        # Start with the dictionary with the tabs defined
        # in base class.
        tab_contents = super().initialize_tab_contents()

        # Add your new plot to this dictionary
        new_tab_name = "My plot"
        tab_contents[new_tab_name] = pn.param.ParamMethod(
            self.make_my_plot, loading_indicator=True
        )

        return tab_contents